In [1]:
import tkinter as tk
from tkinter import *
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
convo = pd.read_csv(r'Conversations - Sheet1 (1).csv')
convo.head()

,Sentences,Intent,Responses
0,Good Morning,Greeting,Good morning. How are you today?
1,I am fine/good/great,Greeting,"Oh. great, That is good to hear. How can i hel..."
2,Hello,Greeting,"Hi, how are you today?"
3,Hi,Greeting,"Hey,what's up with you?"
4,Thank you,Greeting,"You are welcome, is there anything you would l..."


In [3]:
bg_gray = '#B7B7B7'
bg_color = '#CAE1FF'
text_colo = 'black'
# font = 'Comic Sans MS 12'#EEE8CD
# font_bold = 'Helvetica 13 bold'

In [4]:
def get_response(text):
    user_speech = [text]
    sentences = list(convo['Sentences']) + user_speech
#     print (sentences)
    bag_of_words = TfidfVectorizer().fit_transform(sentences)
    similarity_scores = cosine_similarity(bag_of_words)
    similarity_scores = similarity_scores[-1][:-1]
    idx = np.argmax(similarity_scores)
    score = np.max(similarity_scores)
    if score >= 0.6:
        row = convo.iloc[idx]
        response = row['Responses']
#         intent = row['Intent']
    else:
        response = """I am sorry,I do not understand.Can you reword your statement. I am still undergoing training"""
#         intent = None
    return (response)

In [5]:
class ChatApp:
    def __init__(self):
        self.window = tk.Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title('RAINY')
        self.window.resizable(width = False, height = False)
        self.window.configure(width = 450, height = 550, bg= bg_color)

        #head 
        head_label = Label(self.window, bg=bg_color, fg = text_colo,
                            text="Chat with Rainy",font=("Arial", 14, "bold"),pady=10)
        head_label.place(relwidth=1)

        #tiny divider
        line = Label(self.window,width=450,bg= bg_gray)
        line.place(relwidth=1, rely=0.07, relheight=0.012)     

        #text widget
        self.text_widget = Text(self.window,width = 25,height=2,bg= bg_color,fg =text_colo,
                                font=("Courier", 10),padx=7,pady=5,wrap=WORD)
        self.text_widget.place(relheight=0.745,relwidth=1,rely=0.08)
        self.text_widget.configure(cursor = "arrow", state=DISABLED)

        #scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1,relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
#         #bar 
#         bar = Scrollbar(self.text_widget)
#         scrollbar.place(relwidth=1,rely=0.974)
#         scrollbar.configure(command=self.text_widget.xview)

        #bottom label
        bottom_label = Label(self.window,bg= bg_gray,height=80)
        bottom_label.place(relwidth=1,rely = 0.825)  

        #message input
        self.msg_entry = Entry(bottom_label,bg ="white",fg= text_colo,font=("Courier", 10))
        self.msg_entry.place(relwidth=0.74,relheight=0.04,rely=0.009,relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>",self._on_enter_pressed) 

        #send button
        send_button  =Button(bottom_label,text="Enter",font=("Courier", 10 ),width= 20,bg= '#CAE1FF',
                            command = lambda:self._on_enter_pressed(None))
        send_button.place(relx = 0.77,rely=0.008,relheight=0.04,relwidth=0.18)                    

    def _on_enter_pressed(self,event):       
        msg = self.msg_entry.get()
        self._insert_message(msg,"You")

    def _insert_message(self,msg,sender):
        if not msg:
            return

        self.msg_entry.delete(0,END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,msg1)
        self.text_widget.configure(state=DISABLED)

        msg2 = f"Rainy: {get_response(msg)}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,msg2)
        self.text_widget.configure(state=DISABLED)


        

if __name__ == "__main__":
    app = ChatApp()
    app.run()